<a href="https://colab.research.google.com/github/RSkrep/Home_Sales/blob/main/Home_Sales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.5.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!pip install py4j
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark


# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [810 kB]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,082 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://security.ubuntu.com/u

In [18]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [20]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

# Add the S3 file to SparkFiles
spark.sparkContext.addFile(url)

# Read the CSV file into a DataFrame
home_sales_df = spark.read.csv("file://" + SparkFiles.get("home_sales_revised.csv"), header=True, inferSchema=True)

# Show the DataFrame schema and first few rows
home_sales_df.printSchema()
home_sales_df.show(5)

root
 |-- id: string (nullable = true)
 |-- date: date (nullable = true)
 |-- date_built: integer (nullable = true)
 |-- price: integer (nullable = true)
 |-- bedrooms: integer (nullable = true)
 |-- bathrooms: integer (nullable = true)
 |-- sqft_living: integer (nullable = true)
 |-- sqft_lot: integer (nullable = true)
 |-- floors: integer (nullable = true)
 |-- waterfront: integer (nullable = true)
 |-- view: integer (nullable = true)

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|

In [31]:
# 2. Create a temporary view of the DataFrame.

home_sales_df.createOrReplaceTempView("home_sales")



In [32]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
from pyspark.sql import functions as F

# Filter the DataFrame for four-bedroom houses
four_bedroom_df = df.filter(df['bedrooms'] == 4)

# Extract the year from the Sale Date column
four_bedroom_df = four_bedroom_df.withColumn('Year', F.year('date'))

# Group by year and calculate the average price
average_price_per_year = four_bedroom_df.groupby('Year').agg(F.avg('Price').alias('Average Price'))

# Round the average price to two decimal places
average_price_per_year = average_price_per_year.withColumn('Average Price', F.round('Average Price', 2))

# Show the result
average_price_per_year.show()



+----+-------------+
|Year|Average Price|
+----+-------------+
|2022|    296363.88|
|2019|     300263.7|
|2020|    298353.78|
|2021|    301819.44|
+----+-------------+



In [33]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
# Filter the DataFrame for three-bedroom houses with three bathrooms
three_bedroom_df = df.filter((df['Bedrooms'] == 3) & (df['Bathrooms'] == 3))

# Group by year built and calculate the average price
average_price_per_year_house_built = three_bedroom_df.groupby('date_built').agg(F.avg('Price').alias('Average Price'))

# Round the average price to two decimal places
average_price_per_year_house_built = average_price_per_year_house_built.withColumn('Average Price', F.round('Average Price', 2))

# Show the result
average_price_per_year_house_built.show()


+----------+-------------+
|date_built|Average Price|
+----------+-------------+
|      2015|     288770.3|
|      2013|    295962.27|
|      2014|    290852.27|
|      2012|    293683.19|
|      2016|    290555.07|
|      2010|    292859.62|
|      2011|    291117.47|
|      2017|    292676.79|
+----------+-------------+



In [34]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?

# Filter the DataFrame for three-bedroom houses with three bathrooms
two_floor_df = df.filter((df['Bedrooms'] == 3) & (df['Bathrooms'] == 3) & (df['floors'] == 2) & (df['sqft_living'] >= 2000) )

# Group by year built and calculate the average price
average_price_two_floor = two_floor_df.groupby('date_built').agg(F.avg('Price').alias('Average Price'))

# Round the average price to two decimal places
average_price_two_floor = average_price_two_floor.withColumn('Average Price', F.round('Average Price', 2))

# Show the result
average_price_two_floor.show()


+----------+-------------+
|date_built|Average Price|
+----------+-------------+
|      2015|    297609.97|
|      2013|    303676.79|
|      2014|    298264.72|
|      2012|    307539.97|
|      2016|     293965.1|
|      2010|    285010.22|
|      2011|    276553.81|
|      2017|    280317.58|
+----------+-------------+



In [39]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.
import time

# Start time for measuring query execution time
start_time = time.time()

# Filter the DataFrame for homes with an average price greater than or equal to $350,000
filtered_df = df.filter(df['Price'] >= 350000)

# Group by view rating and calculate the average price
average_price_per_view = filtered_df.groupby('View').agg(F.avg('Price').alias('Average Price'))

# Round the average price to two decimal places
average_price_per_view = average_price_per_view.withColumn('Average Price', F.round('Average Price', 2))

# Order by descending view rating
average_price_per_view = average_price_per_view.orderBy('View', ascending=False)

# Show the result
average_price_per_view.show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|View|Average Price|
+----+-------------+
| 100|    1026669.5|
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|    1054325.6|
|  94|    1033536.2|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|    1072285.2|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|    1063498.0|
|  81|   1053472.79|
+----+-------------+
only showing top 20 rows

--- 0.8355295658111572 seconds ---


In [40]:
# 7. Cache the the temporary table home_sales.
spark.sql("CACHE TABLE home_sales")

DataFrame[]

In [41]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [42]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.
# Start time for measuring query execution time
start_time_cached = time.time()

# Run the query using the cached data
average_price_per_view_cached = spark.sql("""
    SELECT View, ROUND(AVG(Price), 2) AS `Average Price`
    FROM home_sales
    WHERE Price >= 350000
    GROUP BY View
    ORDER BY View DESC
""")

# Show the result
average_price_per_view_cached.show()

# Measure the runtime for the cached query
print("--- Cached Query Runtime: %s seconds ---" % (time.time() - start_time_cached))

# Compare the runtime with the uncached runtime
print("Cached Runtime Improvement: %s seconds" % (time.time() - start_time - (time.time() - start_time_cached)))




+----+-------------+
|View|Average Price|
+----+-------------+
| 100|    1026669.5|
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|    1054325.6|
|  94|    1033536.2|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|    1072285.2|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|    1063498.0|
|  81|   1053472.79|
+----+-------------+
only showing top 20 rows

--- Cached Query Runtime: 0.47021031379699707 seconds ---
Cached Runtime Improvement: 6.471837520599365 seconds


In [43]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data

# Partition the DataFrame by the "date_built" field
home_sales_df.write.partitionBy("date_built").format("parquet").save("date_built_parquet_data")



In [44]:
# 11. Read the parquet formatted data.
parquet_df = spark.read.parquet("date_built_parquet_data")

In [45]:
# 12. Create a temporary table for the parquet data.
parquet_df.createOrReplaceTempView("parquet_temp_view")

In [46]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.
# Start time for measuring query execution time
start_time_parquet = time.time()

# Run the query using the Parquet DataFrame
average_price_per_view_parquet = spark.sql("""
    SELECT View, ROUND(AVG(Price), 2) AS `Average Price`
    FROM parquet_temp_view
    WHERE Price >= 350000
    GROUP BY View
    ORDER BY View DESC
""")

# Show the result
average_price_per_view_parquet.show()

# Measure the runtime for the Parquet query
print("--- Parquet Query Runtime: %s seconds ---" % (time.time() - start_time_parquet))

# Compare the runtime with the cached runtime
print("Cached Runtime Improvement: %s seconds" % (time.time() - start_time - (time.time() - start_time_parquet)))


+----+-------------+
|View|Average Price|
+----+-------------+
| 100|    1026669.5|
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|    1054325.6|
|  94|    1033536.2|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|    1072285.2|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|    1063498.0|
|  81|   1053472.79|
+----+-------------+
only showing top 20 rows

--- Parquet Query Runtime: 1.8067195415496826 seconds ---
Cached Runtime Improvement: 167.32002902030945 seconds


In [47]:
# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable("home_sales")


In [48]:
# 15. Check if the home_sales is no longer cached

is_cached = spark.catalog.isCached("home_sales")
print("Is home_sales cached? ", is_cached)


Is home_sales cached?  False
